In [1]:
import overpy
import asyncio
import uvicorn
from typing import List
from fastapi import FastAPI
from pydantic import BaseModel
from uvicorn import Config, Server
        
class GeoData:

    def __init__(self):
        self.api = overpy.Overpass()
    
    def loadByBounds(self, bounds=""):
        self.api = overpy.Overpass()
        result = self.api.query(f"""
                                node(poly:"{bounds}");
                                out body;
                                """)
        data = []
        for node in result.nodes:
            location = dict(name = node.tags.get("name", "n/a"), lat = "%f" % (node.lat), lng = "%f" % (node.lon))
            data.append(location)
        return data


app = FastAPI()

@app.get("/data/")
def fetchDataByBounds(bounds: str):
    geodata = GeoData();
    result = geodata.loadByBounds(bounds)
    return {"result": result}


if __name__ == "__main__":
    config = Config(app)
    server = Server(config=config)
    loop = asyncio.get_running_loop()
    loop.create_task(server.serve())
        

INFO:     Started server process [7384]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:55088 - "GET /data/?bounds=7.252371998643707%205.210173016535857%207.249902815313859%205.206793433177093%207.249083298888855%205.212876683222869%207.251712132010546%205.214646941172698%207.252371998643707%205.210173016535857 HTTP/1.1" 200 OK
